## 30. ベイズ確率論

### <font color = blue>**1.** </font> ベイズ推定

#### <font color = green> **1.1.** </font> pdf教材より

コインの表裏など、結果が２値となる試行の確率を考える

In [ ]:
from scipy.special import beta
from math import log

In [ ]:
N, n = 100, 40

In [ ]:
def L(a, b, NL=N, nL=n):
  return beta(a+nL, b+NL-nL) / beta(a, b)

In [ ]:
def LL(a, b, NLL=N, nLL=n):
  return log(beta(a+nLL, b+NLL-nLL)) - log(beta(a, b))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
a_max, a_min, a_step = 20, 0.1, 0.1

In [ ]:
b_max, b_min, b_step = 20, 0.1, 0.1

In [ ]:
a = [k*a_step + a_min for k in range(int((a_max - a_min) / a_step))]

In [ ]:
b = [k*b_step + b_min for k in range(int((b_max - b_min) / b_step))]

In [ ]:
l = [
     [L(a[m], b[k]) for m in range(int((a_max - a_min) / a_step))]
     for k in range(int((b_max - b_min) / b_step))
     ]

In [ ]:
x, y = np.meshgrid(a, b)

In [ ]:
plt.figure(figsize=(8,8))
plt.contourf(x, y, l, levels=20, cmap='RdYlGn')
plt.show()

In [ ]:
alpha, epsilon = 1e2, .1

In [ ]:
K = 1e5
r_list = [5,]

In [ ]:
for i in range(int(K)):
  r_list += [r_list[-1] + LL(alpha, (r_list[-1] + epsilon)*alpha) - LL(alpha, r_list[-1]*alpha)]

r_list[-1]

In [ ]:
plt.plot(r_list)
plt.xlim(-10, 100)
plt.show()

In [ ]:
l_list = [L(a_step*i, r_list[-1]*a_step*i) for i in range(int(K))]
plt.plot(l_list)
plt.show()

### <font color = blue>**2.** </font> 単純ベイズ（Naive Bayes）

https://scikit-learn.org/stable/modules/naive_bayes.html

#### <font color = green> **2.1.** </font> 公式サンプルコードより

あやめの分類

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt

In [ ]:
X, y = load_iris(return_X_y=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
colors = ['navy', 'turquoise', 'darkorange']

for color, i in zip(colors, [0, 1, 2]):
  plt.scatter(X_train[y_train == i, 2], X_train[y_train == i, 3], color=color, alpha=.8)
plt.show()

In [ ]:
for color, i in zip(colors, [0, 1, 2]):
  plt.scatter(X_test[y_test == i, 2], X_test[y_test == i, 3], color=color, alpha=.8)
plt.show()

In [ ]:
gnb = GaussianNB()

In [ ]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(121)
plt.title("test", fontsize=16)
for color, i in zip(colors, [0, 1, 2]):
  plt.scatter(X_test[y_test == i, 2], X_test[y_test == i, 3], color=color, alpha=.8)

plt.subplot(122)
plt.title("predict", fontsize=16)
for color, i in zip(colors, [0, 1, 2]):
  plt.scatter(X_test[y_pred == i, 2], X_test[y_pred == i, 3], color=color, alpha=.8)

plt.show()

#### <font color = green> **2.2.** </font> pdf教材より

あやめの分類

In [ ]:
from sklearn.datasets import load_iris
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
iris = load_iris()

In [ ]:
iris_target = iris.target

In [ ]:
### 使用する説明変数を2つ指定
iris_data = iris.data[:, 2:4]

In [ ]:
sc = preprocessing.StandardScaler()

In [ ]:
sc.fit(iris_data)

In [ ]:
iris_data2 = sc.transform(iris_data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_data2, iris_target, random_state=0)

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(X_train, y_train)

In [ ]:
gnb.score(X_test, y_test)

In [ ]:
y_pred = gnb.predict(X_test)

In [ ]:
X_max, X_min, X_step = 3, -3, 0.01

In [ ]:
meshnum = int((X_max - X_min)/X_step) + 1

In [ ]:
X_axis = [X_min + i*X_step for i in range(meshnum)]

In [ ]:
X_plane = np.array([
                    [
                     [X_min + i*X_step, X_min + k*X_step]  for i in range(meshnum)
                     ] for k in range(meshnum)
                     ])

In [ ]:
Z = np.array([gnb.predict(X_plane[i]) for i in range(meshnum)])

In [ ]:
plt.figure(figsize=(8,8))
plt.contourf(X_axis, X_axis, Z, cmap='Wistia')

color_list = ['red', 'green', 'blue']
for color, i in zip(color_list, [0, 1, 2]):
  plt.scatter(X_train[y_train == i, 0], X_train[y_train == i, 1], color=color, alpha=1, s=6)

plt.show()

### <font color = blue>**3.** </font> エビデンス近似 : Bayesian Ridge Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html

#### <font color = green> **3.1.** </font> 公式サンプルコードより

正弦波で手動生成

https://scikit-learn.org/stable/auto_examples/linear_model/plot_bayesian_ridge_curvefit.html

Author: Yoshihiro Uchida <nimbus1after2a1sun7shower@gmail.com>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import BayesianRidge

In [ ]:
def func(x):
  return np.sin(2*np.pi*x)

In [ ]:
# #############################################################################
# Generate sinusoidal data with noise
size = 25
rng = np.random.RandomState(1234)
x_train = rng.uniform(0., 1., size)
y_train = func(x_train) + rng.normal(scale=0.1, size=size)
x_test = np.linspace(0., 1., 100)

In [ ]:
# #############################################################################
# Fit by cubic polynomial
n_order = 3
X_train = np.vander(x_train, n_order + 1, increasing=True)
X_test = np.vander(x_test, n_order + 1, increasing=True)

In [ ]:
# #############################################################################
# Plot the true and predicted curves with log marginal likelihood (L)
reg = BayesianRidge(tol=1e-6, fit_intercept=False, compute_score=True)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

for i, ax in enumerate(axes):
  # Bayesian ridge regression with different initial value pairs
  if i == 0:
    init = [1 / np.var(y_train), 1.]  # Default values
    #init = [1e2, 1]
  elif i == 1:
    init = [1., 1e-3]
  reg.set_params(n_iter=int(1e6), alpha_init=init[0], lambda_init=init[1])

  reg.fit(X_train, y_train)
  ymean, ystd = reg.predict(X_test, return_std=True)

  ax.plot(x_test, func(x_test), color="blue", label="sin($2\\pi x$)")
  ax.scatter(x_train, y_train, s=50, alpha=0.5, label="observation")
  ax.plot(x_test, ymean, color="red", label="predict mean")
  ax.fill_between(x_test, ymean-ystd, ymean+ystd,
                  color="pink", alpha=0.5, label="predict std")
  ax.set_ylim(-1.3, 1.3)
  ax.legend(fontsize=12)

  title = "$\\alpha$_init$={:.2f},\\ \\lambda$_init$={}$".format(init[0], init[1])
  if i == 0:
    title += " (Default)"
  ax.set_title(title, fontsize=14)

  text = "$\\alpha={:.1f}$\n$\\lambda={:.3f}$\n$L={:.1f}$".format(
      reg.alpha_, reg.lambda_, reg.scores_[-1])
  ax.text(0.05, -1.0, text, fontsize=18)

plt.tight_layout()
plt.show()

#### <font color = green> **3.2.** </font> 公式サンプルコードより

正規分布で手動生成

https://scikit-learn.org/stable/auto_examples/linear_model/plot_bayesian_ridge.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.linear_model import BayesianRidge, LinearRegression

In [ ]:
# #############################################################################
# Generating simulated data with Gaussian weights
np.random.seed(0)
n_samples, n_features = 100, 100
X = np.random.randn(n_samples, n_features)  # Create Gaussian data

In [ ]:
# Create weights with a precision lambda_ of 4.
lambda_ = 4.
w = np.zeros(n_features)

In [ ]:
# Only keep 10 weights of interest
relevant_features = np.random.randint(0, n_features, 10)
for i in relevant_features:
  w[i] = stats.norm.rvs(loc=0, scale=1. / np.sqrt(lambda_))

In [ ]:
# Create noise with a precision alpha of 50.
alpha_ = 50.
noise = stats.norm.rvs(loc=0, scale=1. / np.sqrt(alpha_), size=n_samples)

In [ ]:
# Create the target
y = np.dot(X, w) + noise

In [ ]:
# #############################################################################
# Fit the Bayesian Ridge Regression and an OLS for comparison
clf = BayesianRidge(compute_score=True)
clf.fit(X, y)

In [ ]:
# OLS (ordinary least squares)
ols = LinearRegression()
ols.fit(X, y)

In [ ]:
# #############################################################################
# Plot true weights, estimated weights, histogram of the weights, and
# predictions with standard deviations

In [ ]:
# Plotting some predictions for polynomial regression
def f(x, noise_amount):
  y = np.sqrt(x) * np.sin(x)
  noise = np.random.normal(0, 1, len(x))
  return y + noise_amount * noise

In [ ]:
degree = 10
X = np.linspace(0, 10, 100)
y = f(X, noise_amount=0.1)
clf_poly = BayesianRidge()
clf_poly.fit(np.vander(X, degree), y)

In [ ]:
X_plot = np.linspace(0, 11, 25)
y_plot = f(X_plot, noise_amount=0)
y_mean, y_std = clf_poly.predict(np.vander(X_plot, degree), return_std=True)

In [ ]:
lw = 2
plt.figure(figsize=(18, 15))

plt.subplot(221)
plt.title("Weights of the model", fontsize=14)
plt.plot(clf.coef_, color='red', linewidth=lw, label="Bayesian Ridge estimate") #'lightgreen'
plt.plot(w, color='gold', linewidth=lw, label="Ground truth", alpha=0.8)
plt.plot(ols.coef_, color='navy', linestyle='--', label="OLS estimate")
plt.xlabel("Features", fontsize=14)
plt.ylabel("Values of the weights", fontsize=14)
plt.legend(loc="best", prop=dict(size=12))

plt.subplot(222)
plt.title("Histogram of the weights", fontsize=14)
plt.hist(clf.coef_, bins=n_features, color='gold', log=True, edgecolor='black')
plt.scatter(clf.coef_[relevant_features], np.full(len(relevant_features), 5.), color='navy', label="Relevant features")
plt.ylabel("Features", fontsize=14)
plt.xlabel("Values of the weights", fontsize=14)
plt.legend(loc="upper left", prop=dict(size=14))

plt.subplot(223)
plt.title("Marginal log-likelihood", fontsize=14)
plt.plot(clf.scores_, color='navy', linewidth=lw)
plt.ylabel("Score", fontsize=14)
plt.xlabel("Iterations", fontsize=14)

plt.subplot(224)
plt.errorbar(X_plot, y_mean, y_std, color='navy', label="Polynomial Bayesian Ridge Regression", linewidth=lw, alpha=0.5)
plt.plot(X_plot, y_plot, color='gold', linewidth=lw, label="Ground Truth")
plt.ylabel("Output y", fontsize=14)
plt.xlabel("Feature X", fontsize=14)
plt.legend(loc="lower left", prop=dict(size=14))

plt.show()

## 31. 関連ベクトルマシーン（Relevance Vector Machine）

### <font color = blue>**1.** </font> 関連度自動決定（Automatic Relevance Determination）

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ARDRegression.html

#### <font color = green> **1.1.** </font> ボストンの住宅価格でARD

In [ ]:
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ARDRegression
import numpy as np

In [ ]:
boston = load_boston()

In [ ]:
boston_data = boston.data
boston_target = boston.target

In [ ]:
sc = preprocessing.StandardScaler()

In [ ]:
sc.fit(boston_data)

In [ ]:
boston_data2 = sc.transform(boston_data)

In [ ]:
sc.fit(boston_target.reshape(-1,1))

In [ ]:
boston_target2 = sc.transform(boston_target.reshape(-1,1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston_data2, boston_target2, random_state=0)

In [ ]:
y_train2 = y_train.ravel()

In [ ]:
ardr = ARDRegression()

In [ ]:
ardr.fit(X_train, y_train2)

In [ ]:
ardr.score(X_test, y_test)

#### <font color = green> **1.2.** </font> ボストンのARDを自作RVM可

In [ ]:
l = len(X_train)

In [ ]:
import math

def kernel(x_0, x_1, theta_0, theta_1, theta_2, theta_3):
  x = x_0 - x_1
  return theta_0 * math.exp(-theta_1 * np.sum(x*x)/2) + theta_2 + theta_3 * x_0@x_1.T

def phi(X, theta_0=.1, theta_1=.1, theta_2=.1, theta_3=.1):
  l1 = len(X)
  return np.array([[1] + [kernel(X_train[i], X[k], theta_0, theta_1, theta_2, theta_3) for i in range(l)] for k in range(l1)])

In [ ]:
ardr2 = ARDRegression(compute_score=True)

In [ ]:
import time
t1 = time.time()

ardr2.fit(phi(X_train), y_train2)

t2 = time.time()
print(t2 - t1)

In [ ]:
#t1 = time.time()

ardr2.score(phi(X_test), y_test)

#t2 = time.time()
#print(t2 - t1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (18, 9))
X = [i for i in range(l+1)]
plt.bar(X, height=np.abs(ardr2.coef_))
plt.show()

In [ ]:
ardr2.fit(phi(X_train, 1, 1, 1, 1), y_train2)

plt.figure(figsize = (18, 9))
X = [i for i in range(l+1)]
plt.bar(X, height=np.abs(ardr2.coef_))
plt.show()

In [ ]:
ardr2.fit(phi(X_train, 1e-2, .1, 1e5, 1e-3), y_train2)

plt.figure(figsize = (18, 9))
X = [i for i in range(l+1)]
plt.bar(X, height=np.abs(ardr2.coef_))
plt.show()

In [ ]:
ardr2.fit(phi(X_train, 1e-5, 1e-5, 1e7, 1e-5), y_train2)

plt.figure(figsize = (18, 9))
X = [i for i in range(l+1)]
plt.bar(X, height=np.abs(ardr2.coef_))
plt.show()

#### <font color = green> **1.3.** </font> Comparison of relevance vector regression and ARDRegression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.linear_model import ARDRegression
from sklearn_rvm import EMRVR

In [ ]:
# #############################################################################
# Generating simulated data with Gaussian weights

# Parameters of the example
np.random.seed(0)

n_samples, n_features = 100, 100

In [ ]:
# Create Gaussian data
X = np.random.randn(n_samples, n_features)

In [ ]:
# Create weights with a precision lambda_ of 4.
lambda_ = 4.
w = np.zeros(n_features)

In [ ]:
# Only keep 10 weights of interest
relevant_features = np.random.randint(0, n_features, 10)
for i in relevant_features:
  w[i] = stats.norm.rvs(loc=0, scale=1. / np.sqrt(lambda_))

In [ ]:
# Create noise with a precision alpha of 50.
alpha_ = 50.
noise = stats.norm.rvs(loc=0, scale=1. / np.sqrt(alpha_), size=n_samples)

In [ ]:
# Create the target
y = np.dot(X, w) + noise

In [ ]:
# #############################################################################
# Fit the ARD Regression
clf = ARDRegression(compute_score=True)
clf.fit(X, y)

In [ ]:
rvr = EMRVR(kernel="linear")
rvr.fit(X, y)

In [ ]:
# #############################################################################
# Plot the true weights, the estimated weights, the histogram of the
# weights, and predictions with standard deviations
plt.figure(figsize=(12, 10))
plt.title("Weights of the model")
plt.plot(rvr.coef_, color="darkblue", linestyle="-", linewidth=2, label="RVR estimate", alpha=0.6)
plt.plot(clf.coef_, color="yellowgreen", linestyle=":", linewidth=4, label="ARD estimate")
plt.plot(w, color="orange", linestyle="-", linewidth=2, label="Ground truth", alpha=0.5)
plt.xlabel("Features")
plt.ylabel("Values of the weights")
plt.legend(loc=1, fontsize=16)
plt.show()

#### <font color = green> **1.4.** </font> Comparison of ARDRegression and the OLS (ordinary least squares) estimator

https://scikit-learn.org/stable/auto_examples/linear_model/plot_ard.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.linear_model import ARDRegression, LinearRegression

In [ ]:
# #############################################################################
# Generating simulated data with Gaussian weights

# Parameters of the example
np.random.seed(0)
n_samples, n_features = 100, 100

In [ ]:
# Create Gaussian data
X = np.random.randn(n_samples, n_features)

In [ ]:
# Create weights with a precision lambda_ of 4.
lambda_ = 4.
w = np.zeros(n_features)

In [ ]:
# Only keep 10 weights of interest
relevant_features = np.random.randint(0, n_features, 10)
for i in relevant_features:
  w[i] = stats.norm.rvs(loc=0, scale=1. / np.sqrt(lambda_))

In [ ]:
# Create noise with a precision alpha of 50.
alpha_ = 50.
noise = stats.norm.rvs(loc=0, scale=1. / np.sqrt(alpha_), size=n_samples)

In [ ]:
# Create the target
y = np.dot(X, w) + noise

In [ ]:
# #############################################################################
# Fit the ARD Regression
clf = ARDRegression(compute_score=True)
clf.fit(X, y)

In [ ]:
ols = LinearRegression()
ols.fit(X, y)

In [ ]:
# Plotting some predictions for polynomial regression
def f(x, noise_amount):
  y = np.sqrt(x) * np.sin(x)
  noise = np.random.normal(0, 1, len(x))
  return y + noise_amount * noise

In [ ]:
degree = 10
X = np.linspace(0, 10, 100)
y = f(X, noise_amount=1)

In [ ]:
clf_poly = ARDRegression(threshold_lambda=1e5)
clf_poly.fit(np.vander(X, degree), y)

In [ ]:
X_plot = np.linspace(0, 11, 25)
y_plot = f(X_plot, noise_amount=0)
y_mean, y_std = clf_poly.predict(np.vander(X_plot, degree), return_std=True)

In [ ]:
# #############################################################################
# Plot the true weights, the estimated weights, the histogram of the
# weights, and predictions with standard deviations
plt.figure(figsize=(18, 15))

plt.subplot(221)
plt.title("Weights of the model", fontsize=14)
plt.plot(ols.coef_, color='yellowgreen', linestyle=':', linewidth=2, label="OLS estimate")
plt.plot(clf.coef_, color='darkblue', linestyle='-', linewidth=2, label="ARD estimate", alpha=0.8)
plt.plot(w, color='orange', linestyle='-', linewidth=2, label="Ground truth", alpha=0.5)
plt.xlabel("Features", fontsize=14)
plt.ylabel("Values of the weights", fontsize=14)
plt.legend(loc=4, prop=dict(size=12))

plt.subplot(222)
plt.title("Histogram of the weights", fontsize=14)
plt.hist(clf.coef_, bins=n_features, color='navy', log=True)
plt.scatter(clf.coef_[relevant_features], np.full(len(relevant_features), 5.), color='gold', marker='o', label="Relevant features")
plt.ylabel("Features", fontsize=14)
plt.xlabel("Values of the weights", fontsize=14)
plt.legend(loc="best", prop=dict(size=12))

plt.subplot(223)
plt.title("Marginal log-likelihood", fontsize=14)
plt.plot(clf.scores_, color='navy', linewidth=2)
plt.ylabel("Score", fontsize=14)
plt.xlabel("Iterations", fontsize=14)

plt.subplot(224)
plt.errorbar(X_plot, y_mean, y_std, color='navy', label="Polynomial ARD", linewidth=2)
plt.plot(X_plot, y_plot, color='gold', linewidth=2, label="Ground Truth")
plt.ylabel("Output y", fontsize=14)
plt.xlabel("Feature X", fontsize=14)
plt.legend(loc=3, prop=dict(size=12))

plt.show()

### <font color = blue>**2.** </font> ライブラリとサンプルコード

https://sklearn-rvm.readthedocs.io/en/latest/index.html

In [ ]:
!pip install -U sklearn-rvm

#### <font color = green> **2.1.** </font> RVM for regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn_rvm import EMRVR

In [ ]:
def create_toy_data(n=20):
  x = np.linspace(0, 1, n)
  t = np.sin(2 * np.pi * x) + np.random.normal(scale=0.1, size=n)
  return x, t

In [ ]:
x_train, y_train = create_toy_data(n=50)
x = np.linspace(0, 1, 100)

In [ ]:
model = EMRVR(kernel="rbf")
model.fit(x_train[:, None], y_train)

In [ ]:
y, y_std = model.predict(x[:, None], return_std=True)

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(x_train, y_train, facecolor="none", edgecolor="g", label="training")
plt.scatter(x[model.relevance_], y[model.relevance_], s=100, facecolor="none", edgecolor="b", label="relevance vector")
plt.plot(x[:, None], y, color="r", label="predict mean")
plt.fill_between(x, y - y_std, y + y_std, color="pink", alpha=0.2, label="predict std.")
plt.legend(loc="best", fontsize=16)

plt.show()

#### <font color = green> **2.2.** </font> Comparison of relevance vector machine and support vector machine


In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR
from sklearn_rvm import EMRVR

In [ ]:
np.random.seed(8)
rng = np.random.RandomState(0)

In [ ]:
# Generate sample data
X = 4 * np.pi * np.random.random(100) - 2 * np.pi

## sinc : cardinal sine
# sinc(x) = sin(πx) / πx (normalized)
y = np.sinc(X)
y += 0.25 * (0.5 - rng.rand(X.shape[0]))  # add noise

X = X[:, None]

In [ ]:
# Fit SVR
svr = SVR(kernel="rbf", gamma="auto")
stime = time.time()
svr.fit(X, y)
print("Time for SVR fitting: %.3f" % (time.time() - stime))

In [ ]:
# Fit RVR
rvr = EMRVR(kernel="rbf")
stime = time.time()
rvr.fit(X, y)
print("Time for RVR fitting: %.3f" % (time.time() - stime))

In [ ]:
X_plot = np.linspace(-2 * np.pi, 2 * np.pi, 10000)[:, None]

In [ ]:
# Predict using SVR
stime = time.time()
y_svr = svr.predict(X_plot)
print("Time for SVR prediction: %.3f" % (time.time() - stime))

In [ ]:
# Predict using Rvm
stime = time.time()
y_rvr = rvr.predict(X_plot, return_std=False)
print("Time for RVR prediction: %.3f" % (time.time() - stime))

In [ ]:
stime = time.time()
y_rvr, y_std = rvr.predict(X_plot, return_std=True)
print("Time for RVR prediction with standard-deviation: %.3f" % (time.time() - stime))

In [ ]:
# Plot results
fig = plt.figure(figsize=(16, 8))
lw = 2
fig.suptitle("RVR versus SVR", fontsize=20)

plt.subplot(121)
plt.scatter(X, y, marker=".", c="k", label="data")
plt.plot(X_plot, np.sinc(X_plot), color="navy", lw=lw, label="True")

plt.plot(X_plot, y_svr, color="turquoise", lw=lw, label="SVR")
support_vectors_idx = svr.support_
plt.scatter(X[support_vectors_idx], y[support_vectors_idx], s=80, facecolors="none", edgecolors="r", label="support vectors")
plt.ylabel("target", fontsize=16)
plt.xlabel("data", fontsize=16)
plt.legend(loc="best", scatterpoints=1, prop={"size": 12})

plt.subplot(122)
plt.scatter(X, y, marker=".", c="k", label="data")
plt.plot(X_plot, np.sinc(X_plot), color="navy", lw=lw, label="True")

plt.plot(X_plot, y_rvr, color="darkorange", lw=lw, label="RVR")
plt.fill_between(X_plot[:, 0], y_rvr - y_std, y_rvr + y_std, color="darkorange", alpha=0.2)
relevance_vectors_idx = rvr.relevance_
plt.scatter(X[relevance_vectors_idx], y[relevance_vectors_idx], s=80, facecolors="none", edgecolors="r", label="relevance vectors")

plt.xlabel("data", fontsize=16)
plt.legend(loc="best", scatterpoints=1, prop={"size": 12})
plt.show()

#### <font color = green> **2.3.** </font> Simple example

In [ ]:
import numpy as np
from sklearn_rvm import EMRVC

In [ ]:
# General a toy dataset:s it's just a straight line with some Gaussian noise:
n_samples = 100
np.random.seed(0)

In [ ]:
X = np.random.normal(size=n_samples)
y = (X > 0).astype(np.float)

In [ ]:
X[X > 0] *= 4
X += .3 * np.random.normal(size=n_samples)

In [ ]:
X = X[:, np.newaxis]

In [ ]:
# Fit the classifier
clf = EMRVC(kernel="linear")
clf.fit(X, y)

In [ ]:
print(clf.predict(X))
#print(clf.predict_proba(X))
print(clf.score(X, y))

#### <font color = green> **2.4.** </font> Simple example precomputed

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels

from sklearn_rvm import EMRVC

In [ ]:
# General a toy dataset:s it's just a straight line with some Gaussian noise:
n_samples = 100
np.random.seed(0)

In [ ]:
X = np.random.normal(size=n_samples)
y = (X > 0).astype(np.float)

In [ ]:
X[X > 0] *= 4
X += .3 * np.random.normal(size=n_samples)

In [ ]:
X = X[:, np.newaxis]

In [ ]:
K = pairwise_kernels(X)

In [ ]:
# Fit the classifier
clf = EMRVC(kernel="precomputed")
clf.fit(K, y)

In [ ]:
print(clf.predict(K))
#print(clf.predict_proba(K))
print(clf.score(K, y))

#### <font color = green> **2.5.** </font> RVM for classification

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn_rvm import EMRVC

In [ ]:
def create_toy_data():
  x0 = np.random.normal(size=100).reshape(-1, 2) - 1.
  x1 = np.random.normal(size=100).reshape(-1, 2) + 1.
  x = np.concatenate([x0, x1])
  y = np.concatenate([np.zeros(50), np.ones(50)]).astype(np.int)
  return x, y

In [ ]:
x_train, y_train = create_toy_data()

In [ ]:
model = EMRVC(kernel="rbf")
model.fit(x_train, y_train)

In [ ]:
model.

In [ ]:
x0, x1 = np.meshgrid(np.linspace(-4, 4, 100), np.linspace(-4, 4, 100))
x = np.array([x0, x1]).reshape(2, -1).T

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(x_train[:, 0], x_train[:, 1], s=40, c=y_train, marker="x")
plt.scatter(model.relevance_vectors_[:, 0], model.relevance_vectors_[:, 1], s=100, facecolor="none", edgecolor="g")
plt.contourf(x0, x1, model.predict_proba(x)[:, 1].reshape(100, 100), np.linspace(0, 1, 5), alpha=0.2)
plt.colorbar()
plt.xlim(-4, 4)
plt.ylim(-4, 4)

plt.gca().set_aspect("equal", adjustable="box")